<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# <h1 align="center" id="heading">EDA with Walmart Sales</h1>

## Business Objective

Walmart, the retail giant that operates a chain of hypermarkets, wants to understand their weekly sales data, especially the impact from holidays and or big events on the weekly sales data; specifically, Super Bowl, Labor Day, Thanksgiving, and Christmas. In addition, Walmart wants to consider the effect from different macroeconomic/external factors. 

##  ☑️ Learning Objectives
At the end of this session, you will know how to

1. Manipulate data of different types using `pandas`
1. Visualize data with `matplotlib` and `seaborn` to Extract insights 
1. Build a pipeline to preprocess data and fit a simple model using `sklearn`

*Note: if you see code that's unfamiliar to you, look up for the documentation, and try to understand what it does.*

## Pre-Work
1. activate your `conda` environment; replace `py39_12` with the name of your `conda` environment (`conda env list` to see all available environments)
    ```
    conda activate py39_12
    ```
2. install the required packages under the current directory: `{Your Local Directory}/MLE-8/assignments/eda-walmart-sales/`
    ```
    pip install -r requirements.txt
    ```
    *NB: requirements.txt contains the following:*
    numpy 
    pandas
    scipy
    matplotlib 
    seaborn 
    scikit-learn==1.0.2
    sweetviz==2.1.4
    
3. Launch jupyter notebook or jupyter lab
    ```
    jupyter notebook # or jupyter lab
    ```
4. Run notebook [imports](nb/imports.ipynb) and make sure no error messages are displayed when importing the packages. 

## Background
Please review the weekly narrative [here](https://great-yamamomo-5c3.notion.site/Week-4-Data-Engineering-9cd074e33a46401da011cc2d8b2a0d2e)


## Sanity check
*Run notebook imports:*

In [1]:
import numpy as np      
import pandas as pd     

import matplotlib.pyplot as plt  
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# EDA with Walmart Sales Data

## Business Objectives

Walmart, the retail giant that operates a chain of hypermarkets, wants to understand their weekly sales data, especially the impact from holidays and or big events on the weekly sales data; specifically, Super Bowl, Labor Day, Thanksgiving, and Christmas. In addition, Walmart wants to consider the effect from different macroeconomic/external factors. 

## Learning Objectives

At the end of this session, you will know how to

1. Manipulate data of different types using `pandas`
1. Visualize data with `matplotlib` and `seaborn` to extract insights 
1. Perform feature engineering
1. Build a pipeline to preprocess data and fit a simple model using `sklearn`

*Note: if you see code that's unfamiliar to you, look up for the documentation, and try to understand what it does.*

## Data Overview

- Original sales data were collected from 45 stores across the United States; yet for this session, you will first inspect data from three stores and later focus on just store 1. 

- Each store is of certain type and size, and there are multiple departments in a store. 

- The dataset has a temporal component, we ignore this mostly in this session and will discuss time series related techniques later in the cohort.

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # allow multiple outputs in a cell
import warnings
warnings.filterwarnings("ignore")

## Task I: Load Data 

Built on top of `numpy`, `pandas` is one of the most widely used tools in machine learning. Its rich features are used for exploring, cleaning, visualizing, and transforming data.  We need to import the library to access all of its features. 

In [3]:
import pandas as pd

Use `pd.read_csv` to read `train_comb.csv` that contains weekly sales, metadata, and macroeconomic features from three stores into a `pd.DataFrame`.

In [4]:
filepath = '../dat/train_comb.csv'
data = pd.read_csv(filepath) 

Verify that the data is loaded correctly by running `data.head(3)` to see the first few row ( AVOID printing out the entire DataFrame, i.e., `data` or `print(data)`; it might be trivial for small dataset but it can crash your kernel when the dataset is big and slow down the initial data exploration process ).

In [5]:
# YOUR CODE HERE
data.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315
5,1,1,2010-03-12,21043.39,False,57.79,2.667,NaN,NaN,NaN,NaN,NaN,211.380643,8.106,A,151315
6,1,1,2010-03-19,22136.64,False,54.58,2.720,NaN,NaN,NaN,NaN,NaN,211.215635,8.106,A,151315
7,1,1,2010-03-26,26229.21,False,51.45,2.732,NaN,NaN,NaN,NaN,NaN,211.018042,8.106,A,151315
8,1,1,2010-04-02,57258.43,False,62.27,2.719,NaN,NaN,NaN,NaN,NaN,210.820450,7.808,A,151315
9,1,1,2010-04-09,42960.91,False,65.86,2.770,NaN,NaN,NaN,NaN,NaN,210.622857,7.808,A,151315


In [6]:
data.tail(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
30980,13,99,2012-08-03,90.02,False,81.99,3.512,48280.91,73.92,98.64,13485.99,7765.27,130.737871,5.765,A,219622
30981,13,99,2012-08-10,169.80,False,81.69,3.509,6249.06,70.38,51.09,3258.43,7070.04,130.756161,5.765,A,219622
30982,13,99,2012-08-17,0.08,False,79.40,3.545,5063.08,6.76,40.96,2344.61,5594.85,130.790968,5.765,A,219622
30983,13,99,2012-08-24,90.06,False,77.37,3.582,9360.80,71.92,179.51,8290.49,6291.99,130.838161,5.765,A,219622
30984,13,99,2012-08-31,30.04,False,79.18,3.624,21327.53,50.44,88.44,9444.60,3872.19,130.885355,5.765,A,219622
30985,13,99,2012-09-07,10.10,True,70.65,3.689,17452.15,18.74,69.23,4472.17,5180.78,130.932548,5.765,A,219622
30986,13,99,2012-09-14,0.17,False,68.55,3.749,13208.43,NaN,28.06,1300.16,10484.98,130.977667,5.765,A,219622
30987,13,99,2012-09-21,59.83,False,67.96,3.821,10671.71,141.83,46.00,2465.37,12372.29,131.010333,5.765,A,219622
30988,13,99,2012-10-05,1130.00,False,61.79,3.815,6607.14,NaN,50.60,3943.91,7700.91,131.075667,5.621,A,219622
30989,13,99,2012-10-12,210.00,False,55.10,3.797,1927.15,NaN,9.72,518.97,11269.24,131.108333,5.621,A,219622


In [9]:
# look at date range of data
# check min/max date (as strings):
print(f"Min date (as string): {data['Date'].min()}\n"
      f"Max date (as string): {data['Date'].max()}\n"
     )

# double-check by sorting data by 'Date' and looking at head/tail:
data_by_date = data.sort_values(by='Date')
data_by_date.head(10)
data_by_date.tail(10)

Min date (as string): 2010-02-05
Max date (as string): 2012-10-26



,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
3688,1,28,2010-02-05,1085.29,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
27363,13,55,2010-02-05,23069.69,False,31.53,2.666,NaN,NaN,NaN,NaN,NaN,126.442065,8.316,A,219622
20188,4,97,2010-02-05,38214.05,False,43.76,2.598,NaN,NaN,NaN,NaN,NaN,126.442065,8.623,A,205863
20331,4,98,2010-02-05,12899.52,False,43.76,2.598,NaN,NaN,NaN,NaN,NaN,126.442065,8.623,A,205863
3545,1,27,2010-02-05,2293.00,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
20516,13,1,2010-02-05,46761.90,False,31.53,2.666,NaN,NaN,NaN,NaN,NaN,126.442065,8.316,A,219622
20659,13,2,2010-02-05,75275.87,False,31.53,2.666,NaN,NaN,NaN,NaN,NaN,126.442065,8.316,A,219622
3402,1,26,2010-02-05,11737.12,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
30375,13,95,2010-02-05,125740.33,False,31.53,2.666,NaN,NaN,NaN,NaN,NaN,126.442065,8.316,A,219622


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
9920,1,96,2012-10-26,36851.92,False,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,223.444251,6.573,A,151315
19186,4,87,2012-10-26,16521.28,False,63.64,3.514,1763.13,88.76,66.76,NaN,7577.14,131.193097,3.879,A,205863
27505,13,55,2012-10-26,17414.45,False,46.97,3.755,10192.49,364.57,150.00,1714.15,5563.92,131.193097,5.621,A,219622
9798,1,95,2012-10-26,117375.38,False,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,223.444251,6.573,A,151315
30803,13,97,2012-10-26,26894.79,False,46.97,3.755,10192.49,364.57,150.00,1714.15,5563.92,131.193097,5.621,A,219622
2400,1,18,2012-10-26,17934.28,False,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,223.444251,6.573,A,151315
19329,4,90,2012-10-26,94014.08,False,63.64,3.514,1763.13,88.76,66.76,NaN,7577.14,131.193097,3.879,A,205863
29230,13,82,2012-10-26,31526.31,False,46.97,3.755,10192.49,364.57,150.00,1714.15,5563.92,131.193097,5.621,A,219622
17741,4,67,2012-10-26,10897.73,False,63.64,3.514,1763.13,88.76,66.76,NaN,7577.14,131.193097,3.879,A,205863
29373,13,83,2012-10-26,3955.55,False,46.97,3.755,10192.49,364.57,150.00,1714.15,5563.92,131.193097,5.621,A,219622


In [7]:
data.shape
data.dtypes
data.sample(10)

(30990, 16)

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
Type             object
Size              int64
dtype: object

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
19802,4,94,2010-12-03,47999.05,False,46.40,2.727,NaN,NaN,NaN,NaN,NaN,126.731333,7.127,A,205863
29765,13,90,2012-02-10,127230.25,True,33.73,3.116,9264.48,8292.73,179.72,18849.21,9083.54,130.384903,6.104,A,219622
10295,4,1,2011-01-28,30977.96,False,40.60,3.022,NaN,NaN,NaN,NaN,NaN,127.580032,6.510,A,205863
8147,1,80,2011-04-29,15630.25,False,72.03,3.810,NaN,NaN,NaN,NaN,NaN,215.627954,7.682,A,151315
14670,4,33,2010-04-30,9278.75,False,53.04,2.787,NaN,NaN,NaN,NaN,NaN,126.380567,7.896,A,205863
2718,1,21,2010-09-10,7966.59,True,78.69,2.565,NaN,NaN,NaN,NaN,NaN,211.495190,7.787,A,151315
12674,4,19,2010-06-11,2835.78,False,78.45,2.668,NaN,NaN,NaN,NaN,NaN,126.111903,7.896,A,205863
7837,1,74,2010-12-31,15132.59,True,48.43,2.943,NaN,NaN,NaN,NaN,NaN,211.404932,7.838,A,151315
18403,4,80,2011-07-08,20453.42,False,84.59,3.469,NaN,NaN,NaN,NaN,NaN,129.112500,5.644,A,205863
9489,1,93,2012-05-18,79074.91,False,70.33,3.630,6154.14,NaN,45.11,1675.49,5508.18,221.742674,7.143,A,151315


❓ Question 1:

Look at the output to get an idea of what each column is and then write a few sentences describing what you notice about the data. You can also use `data.sample(3)` to draw random samples from the data (hints: number of rows and columns, any missing values? data types of the elements? date ranges of the data collected? etc.).

*Acceptable responses include the number of rows and columns in the dataset, the data types of the elements, how many NaNs there are (and perhaps which columns and/or rows tend to have them), the range of values in each column or other descriptive statistics, some commentary on what this data represents, any initial concerns about how you think we should model this data, or any other commentary you would like to add.*

<div class="alert alert-info">
    <h4><b><i>Answer:</i></b></h4>
    <ul>    
    <li>No. rows: 30,990</li>
    <li>No. cols: 16</li>
    <li>The data spans the time period from 2010-02-05 to 2012-10-26.</li>
    <li>It is not clear what the columns 'MarkDown1-5' represent, but they have many missing values 'NaN'.</li>
    <li>It is also unclear what the column 'Type' represents, but entries are of dtype object (string, e.g., 'A').</li>
    <li>Data types of the remaining columns are what one might expect given the type of information in each column (except perhaps for 'Date', in which entries are dtype object (string) as opposed to e.g. datetime).</li>
    <li>The various columns include information that might be relevant to weekly sales, including:
        <ul><li>Macroeconomic indicators e.g. 'CPI' or consumer price index, 'Unemployment' (rate): <i>How much disposable income relative to current prices is available?</i></li>
            <li>'Temperature' (presumably outside ambient, °F): <i>Is it a pleasant day to shop?</i></li>
            <li>'Fuel_price' (presumably USD/gallon of gasoline): <i>How expensive is it to drive to the store?</i></li>
            <li>'Size' (I am guessing the store's floor-space shopping area): <i>Does the store have plenty of space to present a wide selection of items as well as hold on-site backup stock? Is the shopping area spacious and e.g., more relaxed, or tightly packed and perhaps stressful? etc.</i></li>
            <li>'IsHoliday', a boolean indicating whether the specified weekly intervals include a holiday: <i>Does the specified week include e.g., Black Friday or the busy shopping period around Christmas?</i></li>
            <li>A breakdown of weekly sales by specific store and department.<i>...But we do not have the actual location of stores, and for example, associated population density, demographics, household income, road/transportation infrastructure, etc.</i></li>
        </ul>
    </ul>
    
Initial concerns regarding the above data include:
    <ul><li>We are not given an explicit description of what each column represents (and e.g. in what units). I have made some reasonable assumptions above; but as noted, it remains unclear what columns 'MarkDown1-5' and 'Type' represent.</li>
        <li>The absolute magnitude and range/variability of the data differ significantly between columns, suggesting that feature scaling (normalization/standardization) will probably need to be applied. In addition, the 'Date' and 'IsHoliday' columns represent non-continuous, non-quantitative variables.</li>
        <li>We haven't yet checked through all the data in detail for example, to make sure that all the data in each column is of the expected data type or that there are no gross errors in data entry.
    </ul>
</div>

Use `.shape` to inspect the size of the data: sample size and number of features.

In [10]:
# YOUR CODE HERE
data.shape

(30990, 16)

<details>
<summary>Expected Output</summary>
(30990, 16)
</details>

For the following task, we focus on Store `1` only, 

In [ ]:
data_store1 = data[data.Store == 1]

In [ ]:
# let's check out data_store1:
print(f'data_store1 shape is:  {data_store1.shape}')
data_store1.head(10)
data_store1.tail(10)

Retrieve the data from department 9 ( a random choice ) at store 1:

In [ ]:
data_store1_dept9 = data_store1[data_store1.Dept == 9] 

Verify the result using `.head()`, `.shape`.

In [ ]:
data_store1_dept9.head()
data_store1_dept9.shape

<details>
<summary>Expected Output</summary>
(143, 16)
</details>


Visualize one full year of sales. The data came with dates sorted, but we can make sure of it and then visualize the first 52 data  points.

In [ ]:
data_store1_dept9 = data_store1_dept9.sort_values('Date')
data_store1_dept9[['Date', 'Weekly_Sales']].iloc[:52]\
    .set_index('Date').plot(rot=90);

❓ Question 2:

Do you have any hypotheses about the holidays' impact on the sales?

    YOUR ANSWER HERE

**For the purpose of this notebook, we focus on the sales data from Store 1** in DataFrame `df` and is saved in `train_store1.csv`. Let's read in the data.

In [ ]:
df = pd.read_csv("../dat/train-store1.csv")

Extract week, month, and year information from the raw `Date` column to better manipulate the weekly data later. Pandas comes with powerful features to make this step easy. Reference: [tutorial
](https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html). 

First, use `.dtypes` to check the datatype of the `Date` column. What's the difference between `df[['Date']]` and `df['Date']`?.

In [ ]:
# YOUR CODE HERE
df.dtypes
df.shape

<details>
<summary>Expected Output</summary>
Date    object
dtype: object
</details>

##### *Using the property `DataFrame.dtypes`, we can see from the above cell that the datatype of the `Date` column is `object`.*
This means that the entries in the `Date` column are either strings or alternatively, mixed types. To check:

In [ ]:
from pandas.api.types import infer_dtype
infer_dtype(df.Date)

In [ ]:
# df['Date'] indexes column 'Date', returning a pd.Series
df['Date'].dtype
df['Date'].shape

In [ ]:
# However, df[['Date']] indexes/slices returns a single-column data frame having only one column named 'Date':
type(df[['Date']])
df[['Date']].shape

In [ ]:
df.Date=pd.to_datetime(df.Date)

Verify that the `Date` column's datatype has changed as expected:

In [ ]:
df[['Date']].dtypes 

In [ ]:
df['week'] = df.Date.dt.week
df['month'] = df.Date.dt.month 
df['year'] = df.Date.dt.year 

Verify that now there are 19 columns in `df`:

In [ ]:
# YOUR CODE HERE

❓ Question 3:

Last step before we look deeper into the features is to split the data set into training and testing datasets. Discuss: why do we want to perform EDA only on the training data, not the entire dataset? Shouldn't it be the more the better?

    YOUR ANSWER HERE

*The answer should mention data leakage, and / or overfitting*

Split the data into training (80%) and test dataset (20%). Use function `train_test_split` from `scikit-learn` ( a popular library for machine learning in Python ),  and set `random_state` to be 42 for reproducibility ( this is not the best way to do train-test-split due to the temporal nature of the data, however, we will ignore it for now ).

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test =  # YOUR CODE HERE

In [ ]:
print('Original set  ---> ',df.shape,
      '\nTraining set  ---> ',df_train.shape,
      '\nTesting set   ---> ', df_test.shape)

<details>
<summary>Expected Output</summary>

    ```
    Original set  --->  (10244, 19) 
    Training set  --->  (8195, 19) 
    Testing set   --->  (2049, 19)
    ```
</details>

## Task II: Target, Features, and Distributions

We inspect the datatype of column `Date`; now find datatypes for all columns in `df_train` using `.dtypes`:

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected Output</summary>

```
Store                    int64
Dept                     int64
Date            datetime64[ns]
Weekly_Sales           float64
IsHoliday                 bool
Temperature            float64
Fuel_Price             float64
MarkDown1              float64
MarkDown2              float64
MarkDown3              float64
MarkDown4              float64
MarkDown5              float64
CPI                    float64
Unemployment           float64
Type                    object
Size                     int64
week                     int64
month                    int64
year                     int64
dtype: object
```
</details>

Summary statistics provide you with a general understanding of the data. Use method `.describe()`. By default it reports statistics mean, max, min, quantiles for numerical features and counts, unique, mode for categorical features.

In [ ]:
pd.options.display.float_format = "{:,.2f}".format
# YOUR CODE HERE

<details>
<summary>Expected Output</summary>

```
	Store	Dept	Weekly_Sales	Temperature	Fuel_Price	MarkDown1	MarkDown2	MarkDown3	MarkDown4	MarkDown5	CPI	Unemployment	Size	week	month	year
count	8,195.00	8,195.00	8,195.00	8,195.00	8,195.00	2,931.00	2,424.00	2,878.00	2,931.00	2,931.00	8,195.00	8,195.00	8,195.00	8,195.00	8,195.00	8,195.00
mean	1.00	44.65	21,865.28	68.19	3.22	8,045.43	2,961.55	1,236.83	3,683.59	5,023.69	216.00	7.61	151,315.00	25.89	6.47	2,010.96
std	0.00	29.95	27,970.00	14.16	0.43	6,484.49	8,032.30	7,830.99	5,849.69	3,303.07	4.33	0.38	0.00	14.19	3.25	0.80
min	1.00	1.00	-863.00	35.40	2.51	410.31	0.50	0.25	8.00	554.92	210.34	6.57	151,315.00	1.00	1.00	2,010.00
25%	1.00	20.00	3,502.09	57.79	2.76	4,039.39	40.48	6.00	577.14	3,127.88	211.57	7.35	151,315.00	14.00	4.00	2,010.00
50%	1.00	38.00	10,357.32	69.64	3.29	6,154.14	137.86	30.23	1,822.55	4,325.19	215.46	7.79	151,315.00	26.00	6.00	2,011.00
75%	1.00	72.00	31,647.36	80.48	3.59	10,121.97	1,569.00	101.64	3,639.42	6,222.25	220.64	7.84	151,315.00	38.00	9.00	2,012.00
max	1.00	99.00	203,670.47	91.65	3.91	34,577.06	46,011.38	55,805.51	32,403.87	20,475.32	223.44	8.11	151,315.00	52.00	12.00	2,012.0
```
</details>

❓ Question 4:

Inspect the output, what are some of your observations?

    YOUR ANSWER HERE

Are there any missing values? Use `.isna()` and `.sum()` to show the number of missing values from each column.

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected Output</summary>

```
Store              0
Dept               0
Date               0
Weekly_Sales       0
IsHoliday          0
Temperature        0
Fuel_Price         0
MarkDown1       5264
MarkDown2       5771
MarkDown3       5317
MarkDown4       5264
MarkDown5       5264
CPI                0
Unemployment       0
Type               0
Size               0
week               0
month              0
year               0
dtype: int64
```
</details>

What do you think the target variable is in this problem? Assign the column name to `target` for later use.

In [ ]:
target = # YOUR CODE HERE

Visualize the distribution of target variable using `distplot()` from library `seaborn` ( Why seaborn? Check out a comparison between Matplotlib and Seaborn [here](https://analyticsindiamag.com/comparing-python-data-visualization-tools-matplotlib-vs-seaborn/) ). Anything here you observe but the output from `.describe` does not make obvious? Does it follow a normal distribution?

In [ ]:
import seaborn as sns
sns.distplot(df_train[target],bins=10);

Notice that there exists nonpositive weekly sales. How many of rows are there that the weekly sales are negative or 0?

In [ ]:
(df_train[target] <= 0).sum() # Expected Output: 13

What percentage is the negative and zero sales?

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected Output</summary>

`0.0015863331299572911` or `0.16%`
</details>

After communicating your findings, the stakeholders confirm that you can remove these data entries for now and they are launching an investigation by analysts and data engineers. 

Now remove them from the training dataset.

In [ ]:
mask = df_train[target] > 0
df_train = # YOUR CODE HERE
df_train.shape # Expected Output: (8182, 19)

Let's move on to features. 

Though almost all the come through as numerical, should they all be treated as numerical features? Let's inspect the number of unique values: 

In [ ]:
[(col, df[col].nunique())for col in df_train.columns]

`Temperature`, `CPI`, `Unemployment`, `Fuel_Price` are continuous. Those tie to the second business objective. Let us put these four into a list and store it in `external_factors`. From earlier, we noticed that `MarkDownx` columns contain some missing values, we will treat them in a later task. 

In [ ]:
external_factors = ['Temperature','CPI','Unemployment', 'Fuel_Price']

Visualize Temperature in a box plot, what do you think is the advantage of a box plot over a histogram? You can use `pd.DataFrame.boxplot()`, set the figure size as (6, 4), and turn off the grid.

❓ Question 5:

Visualize Temperature in a box plot, what do you think the advantage of a box plot over histogram? 

HINT: You can use `pd.DataFrame.boxplot()`, set the figure size as (6, 4), and turn off the grid.

`YOUR ANSWER HERE`

In [ ]:
# YOUR CODE HERE

Let's visualize all four numerical features in both density plot and box plot. Note any observations.

In [ ]:
import matplotlib.pyplot as plt
print('\033[1mNumeric Features Distribution'.center(100))

figsize = (12, 4)

n=len(external_factors)
colors = ['g', 'b', 'r', 'y', 'k']

# histogram
plt.figure(figsize=figsize)
for i in range(len(external_factors)):
    plt.subplot(1,n,i+1)
    sns.distplot(df_train[external_factors[i]],
                 bins=10, 
                 color = colors[i])
plt.tight_layout();

# boxplot
plt.figure(figsize=figsize)
for i in range(len(external_factors)):
    plt.subplot(1,n,i+1)
    df_train.boxplot(external_factors[i], grid=False)
plt.tight_layout();

We will investigate the impacts of the external factors later. Now let's scan through the other features.

`Store`, `Type`, and `Size` each has only one unique value, offering no information, we can safely ignore them. 

We extracted `year`, `month`, and `week` from  `Date`, thus `Date` is redundant; but it is easy to find the date range in the training dataset using `Date`:

In [ ]:
df_train['Date'].min(), df_train['Date'].max() # Expected Output: (Timestamp('2010-02-05 00:00:00'), Timestamp('2012-10-26 00:00:00'))

Our training data ranges from 5th of February 2010 to 26th of October 2012.

It makes more sense to treat `year`, `month`, `week` as categorical, more accurately ordinal; and the boolean feature `IsHoliday` can be considered as categorical, so can `Dept`. Let's put these column names into a list `categoricalFeatures`.

In [ ]:
categoricalFeatures = ['year','month','week','IsHoliday', 'Dept']

For the categorical features, we are more interested in the frequency of each value, use `pd.Series.value_counts` to see how many rows where `IsHoliday` is true and false respectively ( Data imbalance is the norm ).

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected Output</summary>

```
False    7586
True      596
Name: IsHoliday, dtype: int64
```
</details>

Visualize the distribution of `month`; use `sns.countplot()`.

In [ ]:
# YOUR CODE HERE

In [ ]:
#Visualising the categorical features 

print('\033[1mVisualising Categorical Features:'.center(100))

plt.figure(figsize=(12,12))

for i in range(len(categoricalFeatures)):
    plt.subplot(6,1,i+1)
    sns.countplot(df_train[categoricalFeatures[i]])
plt.tight_layout();

❓ Question 6: 

Discuss with your pair programming partner: There is less data in 2012 than the previous two years, did the sale drop from previous years? Does it affect what we see in the plots for month and week? Does the plot below clarify it to some degree?

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_train, x="week", y="Weekly_Sales",  style='year');

`YOUR ANSWER HERE`

## Task III: Impact from Holidays

The first business objective is to understand the impact of holidays on weekly sales. 

There is a flag provided for us: `IsHoliday`, let's calculate the average weekly sales for holiday weeks and non-holiday weeks, respectively. For this, we will use `.groupBy` and `.mean()`. Are holiday sales higher?

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected Output</summary>

```
IsHoliday
False   21,756.05
True    23,737.05
Name: Weekly_Sales, dtype: float64
```
</details>


But we would like to understand it at more granular level, remember [Simpson's paradox](https://en.wikipedia.org/wiki/Simpson's_paradox)? To save some time,  date mapping are identified for the training data

- Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12
- Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12
- Thanksgiving: 26-Nov-10, 25-Nov-11
- Christmas: 31-Dec-10, 30-Dec-11

We create a flag for each holiday to help you analyze weekly sale by each holiday type

In [ ]:
superbowl_mask = df_train['Date'].isin(['2010-02-12', '2011-02-11', '2012-02-10'])
laborday_mask = df_train['Date'].isin(['2010-09-10', '2011-09-09','2012-09-07'])
thanksgiving_mask = df_train['Date'].isin(['2010-11-26', '2011-11-25'])
christmas_mask = df_train['Date'].isin(['2010-12-31', '2011-12-30'])

In [ ]:
df_train['superbowl'] = superbowl_mask
df_train['laborday'] = laborday_mask
df_train['thanksgiving'] =thanksgiving_mask
df_train['christmas'] = christmas_mask

Run the next cell to see 1) how many weekly sales fell on Christmas (does it make sense? what did we not account for?) 2) what is the average weekly sales stratified by whether it is Christmas week or not?

In [ ]:
df_train.groupby(['christmas'])\
        .agg(count = ('christmas', 'size'), 
             avg_weekly_sales= ('Weekly_Sales','mean'))

Perform the same for the other three holidays:

In [ ]:
holidays = ['superbowl', 'laborday', 'thanksgiving', 'christmas']
for holiday in holidays:
    summary_stats = df_train.groupby([holiday])\
        # YOUR CODE HERE
        # YOUR CODE HERE
    print(summary_stats)
    print()

<details>
<summary>Expected Output</summary>

```
           count  avg_weekly_sales
superbowl                         
False       8001         21,845.80
True         181         24,311.98

          count  avg_weekly_sales
laborday                         
False      8007         21,884.35
True        175         22,632.78

              count  avg_weekly_sales
thanksgiving                         
False          8067         21,813.97
True            115         27,959.84

           count  avg_weekly_sales
christmas                         
False       8057         21,921.06
True         125         20,565.56
```
</details>


Without hypothesis testing and by only eyeballing, it seems like Super Bowl and Thanksgiving has a positive impact on the weekly sales for Store 1 in this training dataset. 
Discuss with your teammate, are you surprised that during Christmas, sales at Walmart do not go up? Holiday effect, if causal, happened most during Thanksgiving weeks, is this something you expected?

We have been ignoring `Dept`, let's take a look at the plot below showing the weekly sales by department in 2011. 

In [ ]:
plt.figure(figsize=(10,4))
sns.scatterplot(data=df_train[df_train.year==2011], x = 'Dept', y= target, hue='IsHoliday');

Dept 72 has a very unusual high weekly sales during the holiday week, but we will need more data to understand if this is data issue, outlier, or special event. 

## Task IV: Visualize Relationship between Macroeconomic & External Factors and Sales

In [ ]:
sns.lineplot(data=df_train, x="Fuel_Price", y="Weekly_Sales");

In [ ]:
sns.lineplot(data=df_train, x="Temperature", y="Weekly_Sales");

In [ ]:
sns.lineplot(data=df_train, x="CPI", y="Weekly_Sales");

In [ ]:
sns.lineplot(data=df_train, x="Unemployment", y="Weekly_Sales");

By eyeballing, do you find strong evidence that those are correlated with Walmart's weekly sales? Do you think `lineplot` is an appropriate plot for this?

Lastly, we calculate the spearman correlations among target and external factors and verify that there is no strong linear correlation between the target variable and these features.

In [ ]:
plt.figure(figsize=(6, 6))
df_train_reduced = df_train[[target] + external_factors]
corr = df_train_reduced.corr(method='spearman')
heatmap = sns.heatmap(corr.sort_values(by=target, ascending=False),
                      vmin=-1, vmax=1, annot=True, fmt='.1g', cmap='BrBG')
heatmap.set_title('Features Correlating with Sales Price', fontdict={'fontsize':12}, pad=16);

## Task V: Feature Engineering

"*Feature Engineering encapsulates various data engineering techniques such as selecting relevant features, handling missing data, encoding the data, and normalizing it. It is one of the most crucial tasks and plays a major role in determining the outcome of a model.*" [Ref](https://www.analyticsvidhya.com/blog/2021/10/a-beginners-guide-to-feature-engineering-everything-you-need-to-know/).

One part of feature engineering is to create new features from given data, like `thanksgiving` column earlier was derived from `Date`. Common techniques for tabular data include to add summary statistics of the numerical features such as mean and standard deviation, to create new features from the interaction of multiple features, etc. In this task, however, we will work on handling missing data, normalizing numerical features, and encoding categorical features. 

First, missing data. Missing value treatment is crucial, yet not trivial. Take a read on [Tackling Missing Value in Dataset](https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/) for detailed explanation. Features with nulls or wrong values (e.g., negative fuel price) needs to be imputed or removed. 

- Do you want to keep the features with missing value? Discuss the trade offs
- If answer to the first question is yes, then how do you want to impute them? Discuss the trade offs

From ealier steps, we observed that only the markdown columns contain missing values, yet we do not have more information on what those values are for.  

In [ ]:
df_train.columns[df_train.isna().sum() != 0]

For each column, find out the percentage of the data is missing

In [ ]:
md_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
for col in ['MarkDown'+str(i) for i in range(1,6)]:
    perc_missing =  # YOUR CODE HERE; perc_missing:float
    print (f'{col}: {perc_missing:.0%} is missing')

<details>
<summary>Expected Output</summary>

```
MarkDown1: 64% is missing
MarkDown2: 70% is missing
MarkDown3: 65% is missing
MarkDown4: 64% is missing
MarkDown5: 64% is missing
```
</details>


The majority of the markdown fields are missing. This is where, again, we need to communicate with the stakeholders to understand what the data measure, how the data was collected and then determine our strategy from there. Since we want to understand the impacts of `MarkDownx` on weekly sales, we will keep the features and impute the missing values.  We have learned that there are tradeoffs with how we treat missing values and that our choice of imputation can be significantly impacted by extreme values and the amount of the missing data.  We choose to impute with the median here to mitigate these negative impacts. Use `.fillna()` to impute the missing values.

In [ ]:
# YOUR CODE HERE # this works for smaller dataset

In [ ]:
(df_train.isna().sum() != 0).sum() # sanity check: 0

Visualize the distributions for those markdown fields after imputations, are they normal?

In [ ]:
plt.figure(figsize=figsize)
for i in range(len(md_cols)):
    plt.subplot(1,len(md_cols),i+1)
    sns.distplot(df_train[md_cols[i]],
                 hist_kws=dict(linewidth=2),
                 bins=10, 
                 color = colors[i])
plt.tight_layout();

Note that missing values are different from outliers. Outliers, on the other hand, are feature values that are rare in nature. They can unncessarily skew the data and causes problem for modeling. Outlier treatment involves removing or imputing such values. One popular approach to identify outliers is IQR; that is, data points that lie 1.5 times of IQR above Q3 (third quartile) and below Q1 (first quartile) are outliers. Take a read on [Detecting and Treating Outliers](https://www.analyticsvidhya.com/blog/2021/05/detecting-and-treating-outliers-treating-the-odd-one-out/). We will leave it as an optional exercise for you to identify outliers using IQR, and replace the outliers with the median.

Now let's see how we normalize the data. For numerical features it means scaling the features to be of similar range. This step is crucial for machine learning algorithms that calculate distances between data (e.g., read [The Importance of Feature Scaling](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html). 

For this task, of the external features, let's keep Temperature since it is the most linearly correlated with the target variable, though very weak and negative ( feature selection ). In addition, we include one markdown field. Since neither seems to follow normal distributions, it is safer to use `MinMaxScaler` from `sklearn.preprocessing` to transform features by scaling each feature to a given range (See discussion on [Normalization vs Standardization](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

numericalFeatures = ['Temperature', 'MarkDown1']
df_train_num = df_train[numericalFeatures]

In [ ]:
df_train_num.describe() # Check the summary statistics

Instantiate a MinMaxScaler and fit using `df_train_num`:

In [ ]:
scaler = # YOUR CODE HERE

Now transform training data `df_train_num` and store the resulting nparray in `train_norm`:

In [ ]:
train_norm = scaler.transform(df_train_num) 

Verify that both columns now have minimum 0 and maximum 1. 

In [ ]:
pd.DataFrame(train_norm, columns=df_train_num.columns).describe()

In [ ]:
# Expected Output:

Let's turn to categorical fatures. So far most, if not all Python packages for modeling do not accept strings as input; thus encoding the categorical value to numerical value is a necessary step. Here, let's apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) on `Dept` and `IsHoliday`:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categoricalFeatures = ['Dept', 'IsHoliday']
df_train_cat = df_train[categoricalFeatures]
ohe = OneHotEncoder(handle_unknown='ignore',sparse = False).fit(df_train_cat)

Transform the categorical features using one hote encoding `ohe`. 

In [ ]:
train_ohe = # YOUR CODE HERE

In [ ]:
train_ohe.shape, df_train_cat.shape # Expected Output: ((8182, 79), (8182, 2))

The number of columns explodes from 2 to 79. 

Lastly we merge the processed numerical features with the processed categorical features using `hstack` in `numpy`:

In [ ]:
import numpy as np
X_train = np.hstack([train_norm, train_ohe])

In [ ]:
X_train.shape # sanity check: (8182, 81)

What about the test data? Yes you need to apply the same treatments. We spare some copy + paste + edit and see how this can be done when we introduce `pipeline` next. 

## Task VI: Pipeline

Even with less than 20 features in our dataset, there are many many possibilities that you can preprocessing the data. There is no one-fits-all approach; often you will find yourself experimenting with many combinations to achieve better modelling performance: Should I apply normalization or standardization? Do I remove the outliers or should I impute them? Do I impute the missing values with median or mean or 0? Answers to many of these questions is "It depends." (Have you heard [Graduate Student Descent](https://sciencedryad.wordpress.com/2014/01/25/grad-student-descent/)?) That means trial-and-error and it is not efficient to produce a notebook each time when you need to try something slightly different. You will get lost quickly. Pipeline is one useful tool. 

Not only does Pipeline help streamline the process, keep the code modular, but also reduces the possibility of introducing errors/bugs. In this task, we build the pipeline following the strategies used in the last task, run a simple linear regression model, and print out the model's performance. Note there is minimal code required for you to implement, the key is to understand each step.

To avoid confusion, let's read the data again directly from `train-store1.csv`.

In [ ]:
df = pd.read_csv('../dat/train-store1.csv')

In [ ]:
df.shape

Separating the target `y` from the features `X`:

In [ ]:
X, y = df.drop(columns=target), df[target]

Import `Pipeline` from submodule `sklearn.pipeline`

In [ ]:
from sklearn.pipeline import Pipeline

Now we build a transformer for numerical features following two steps: impute the missing values with the feature median (use `SimpleImputer`), followed by normalization (use `MinMaxScaler`)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
numeric_features = ['CPI', 'MarkDown1']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")), 
    # YOUR CODE HERE
])

For categorical features, we apply one hot encoding `OneHotEncoder` ( there are many other options; see [Scikit-learn documentation](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) ):

In [ ]:
categorical_features = ['Dept', 'IsHoliday']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

Piece the `numeric_transformer` and `categorical_transformer` using `ColumnTransformer`:

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Lastly, let's append the regression model to preprocessing pipeline to complete a full prediction pipeline.

In [ ]:
from sklearn.linear_model import LinearRegression

model = Pipeline(
    steps=[("preprocessor", preprocessor), ("model", LinearRegression())]
)

The pipepline has been built! The rest is to 
- split the data into training and testing sets
- apply the pipeline to the training data
- obtain the prediction performance on testing data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Let's run the prediction!

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print("model score: %.3f" % model.score(X_test, y_test))

Optional: Discuss what type of [Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) strategy you would use to select the features.

## Automating EDA

In this exercise, you have learned the manual way to perform EDA.  Doing EDA manually has the benefits of customization, but is also highly repetitive.  For this reason, a lot of EDA can easily be automated!  In automating our EDA, we can get to know our data more quickly and spend more time on feature engineering and modeling.  Let's check out a library called [SweetViz](https://github.com/fbdesignpro/sweetviz) to see how we can automate EDA! 

In [ ]:
import sweetviz as sv

orig_data_report = sv.analyze(df)
orig_data_report.show_notebook()

1. Click on a feature to tab to explore the feature in more detail.
1. Notice that `SweetViz` calculates the descriptive stats for each feature, along with its missing and duplicate value stats.
1. Notice that `SweetViz` helps to detect numerical vs categorical datatypes.
1. Click on the `ASSOCIATIONS` tab to explore associations/correlations!

### Prefer a browswer experience?

In [ ]:
orig_data_report.show_html('orig_data_report.html', open_browser=True)

### Now let's have a look at a comparison report of our train and test datasets!

In [ ]:
compare_report = sv.compare([X_train, 'Train'], [X_test, 'Test'])
compare_report.show_notebook()

## Note

- EDA, like other parts of machine learning, is an iterative process, NOT linear.
- This analysis is far from being comprehensive; rather it is a starting point. 
- There does not exist one "standard" way to perform EDA. You should always keep business objectives in mind and perform analysis as seen fit. It is one of those skills that grows with lots of practices.

## References

1. Original dataset is from [kaggle: wallmart sales forecast datasets](https://www.kaggle.com/datasets/iamprateek/wallmart-sales-forecast-datasets)
2. Notebook: [craking the walmart sales forecasting challenge](https://www.kaggle.com/code/fernandol/cracking-the-walmart-sales-forecasting-challenge)